<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-dependencies" data-toc-modified-id="Load-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load dependencies</a></span></li><li><span><a href="#Read-csv-files-to-DFs" data-toc-modified-id="Read-csv-files-to-DFs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read csv files to DFs</a></span></li><li><span><a href="#Fill-missing-values" data-toc-modified-id="Fill-missing-values-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fill missing values</a></span></li><li><span><a href="#Add-date-part" data-toc-modified-id="Add-date-part-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add date part</a></span></li><li><span><a href="#Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc." data-toc-modified-id="Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add extra columns (purchased on weekend, monthend, month_diff etc.</a></span></li><li><span><a href="#Aggregate-by-card_id" data-toc-modified-id="Aggregate-by-card_id-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Aggregate by card_id</a></span></li><li><span><a href="#Add-exta-interpreted-columns-on-aggregates" data-toc-modified-id="Add-exta-interpreted-columns-on-aggregates-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Add exta interpreted columns on aggregates</a></span></li><li><span><a href="#Aggregate-on-categories" data-toc-modified-id="Aggregate-on-categories-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate on categories</a></span></li><li><span><a href="#Load-test-&amp;-train-DFs" data-toc-modified-id="Load-test-&amp;-train-DFs-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Load test &amp; train DFs</a></span></li><li><span><a href="#Add-date-part-to-test-&amp;-train-dfs" data-toc-modified-id="Add-date-part-to-test-&amp;-train-dfs-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Add date part to test &amp; train dfs</a></span></li><li><span><a href="#Merge-train-&amp;-test-with-new-&amp;-old-transactions-history" data-toc-modified-id="Merge-train-&amp;-test-with-new-&amp;-old-transactions-history-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Merge train &amp; test with new &amp; old transactions history</a></span></li><li><span><a href="#Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc" data-toc-modified-id="Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Add extra columns like age, total transactions, installments, purchase amount, first buy etc</a></span></li><li><span><a href="#Mark-the-outliers" data-toc-modified-id="Mark-the-outliers-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Mark the outliers</a></span></li><li><span><a href="#Adding-train-days" data-toc-modified-id="Adding-train-days-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Adding train days</a></span></li><li><span><a href="#Target-encode-the-outliers" data-toc-modified-id="Target-encode-the-outliers-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Target encode the outliers</a></span></li></ul></div>

1. Load dependencies
2. Read csv files to dataframes
3. Fill missing values
4. Add date part 
5. Add extra columns (purchased on weekend, monthend, month_diff etc.
6. Aggregate by card_id
7. Aggregate by categories
8. Mark categorical columns
9. Add exta interpreted columns on aggregates
10. Load test & train csvs to dfs
11. Add date part to test & train dfs
12. Merge train & test with new & old transactions history
13. Add extra columns like age, total transactions, installments, purchase amount, first buy etc
14. Mark the outliers
15. Target encode the outliers 
16. Save to feather

### Load dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

### Read csv files to DFs

In [3]:
PATH = '../data/elo/'

In [4]:
files = ['historical_transactions', 'new_merchant_transactions']

In [5]:
hist_trans, new_hist_trans = [pd.read_csv(f'{PATH}{c}.csv') for c in files]

### Fill missing values

In [6]:
def fill_nas_for_transactions_df(df):
    # Fill nas for category_3 with mode
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    return df

In [7]:
dfs = [hist_trans, new_hist_trans]

In [8]:
hist_trans, new_hist_trans = [fill_nas_for_transactions_df(df) for df in dfs]

### Add date part

In [9]:
add_datepart(hist_trans, 'purchase_date', drop=False)
add_datepart(new_hist_trans, 'purchase_date', drop=False)

### Add extra columns (purchased on weekend, monthend, month_diff etc.

In [10]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['category_3'] = df['category_3'].map({'A':0, 'B':1, 'C':2}) 
    return df

In [11]:
hist_trans, new_hist_trans = [add_extra_cols(df) for df in dfs]

In [12]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

In [13]:
hist_trans.to_feather('hist_trans')
new_hist_trans.to_feather('new_hist_trans')

In [14]:
hist_trans = feather.read_dataframe('hist_trans')

In [15]:
new_hist_trans = feather.read_dataframe('new_hist_trans')

### Aggregate by card_id

In [16]:
# hist_trans.head().T

In [17]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 
                   'subsector_id', 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size', 'count']
    for col in ['category_2','category_3']:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_mean'] = ['mean']

    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')

    new_df['purchase_date_diff'] = (new_df['purchase_date_max'] - new_df['purchase_date_min']).dt.days
    new_df['purchase_date_average'] = new_df['purchase_date_diff']/new_df['card_id_size']
    new_df['purchase_date_uptonow'] = (datetime.datetime.today() - new_df['purchase_date_max']).dt.days
    new_df['purchase_date_uptomin'] = (datetime.datetime.today() - new_df['purchase_date_min']).dt.days
    return new_df

In [18]:
# dfs_e = [hist_trans_c_e, new_hist_trans_c_e]

In [19]:
import gc
gc.collect()

0

In [20]:
%time hist_trans_agg, new_hist_trans_agg = [aggregate_by_card_id(df) for df in dfs]

CPU times: user 1min 56s, sys: 15.8 s, total: 2min 11s
Wall time: 1min 16s


### Add exta interpreted columns on aggregates

In [21]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_diff']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_max']).dt.days
    df['repurchase_merchant_rate'] = df['transactions_count']/df['merchant_id_nunique']
    df['merchant_category_repurchase'] = df['merchant_category_id_nunique']/df['merchant_id_nunique']
    df['avg_spend_per_merchant'] = df['purchase_amount_sum']/df['merchant_id_nunique']
    df['avg_trans_per_merchant'] = df['transactions_count']/df['merchant_id_nunique']
    df['avg_spend_per_transaction'] = df['purchase_amount_sum']/df['transactions_count']
    return df

In [22]:
[hist_trans_agg, new_hist_trans_agg] = [add_extra_cols_on_agg(df) for df in [hist_trans_agg, 
                                                                             new_hist_trans_agg]]

In [23]:
hist_trans_agg.to_feather('hist_trans_agg')
new_hist_trans_agg.to_feather('new_hist_trans_agg')

In [24]:
hist_trans_agg = feather.read_dataframe('hist_trans_agg')
new_hist_trans_agg = feather.read_dataframe('new_hist_trans_agg')

### Aggregate on categories

In [25]:
def agg_on_cat(df, category, feature):
    temp_df = df.pivot_table(index='card_id', columns=category, aggfunc={feature: ['sum', 'mean']})
    cols = [category + '_{0[2]}_{0[0]}_{0[1]}'.format(col) for col in temp_df.columns.tolist()]
    temp_df.columns = cols
    return temp_df

In [26]:
def get_cat_agg(df):
    agg_df = agg_on_cat(df, 'category_1', 'purchase_amount')
    agg_df = pd.merge(agg_df, agg_on_cat(df, 'category_2', 'purchase_amount'), on='card_id', how='left')
    agg_df = pd.merge(agg_df, agg_on_cat(df, 'category_3', 'purchase_amount'), on='card_id', how='left')
    agg_df = pd.merge(agg_df, agg_on_cat(df, 'authorized_flag', 'purchase_amount'), on='card_id', how='left')
    return agg_df

In [27]:
# %time hist_trans_agg_cat, new_hist_trans_agg_cat = [get_cat_agg(df) for df in [hist_trans, new_hist_trans]]

In [28]:
# hist_trans_agg_cat.shape, new_hist_trans_agg_cat.shape

### Load test & train DFs

In [29]:
PATH = 'data/elo/'

In [30]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

### Add date part to test & train dfs

In [31]:
add_datepart(train, 'first_active_month', drop=False)
add_datepart(test, 'first_active_month', drop=False)

### Merge train & test with new & old transactions history

In [32]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [33]:
train_df = join_dfs(train, new_hist_trans_agg, left_on='card_id')

In [34]:
train_df = join_dfs(train_df, hist_trans_agg, left_on='card_id', suffix='_old')
# train_df = join_dfs(train_df, hist_trans_agg_cat, left_on='card_id', suffix='_old')

In [35]:
test_df = join_dfs(test, new_hist_trans_agg, left_on='card_id')
test_df = join_dfs(test_df, hist_trans_agg, left_on='card_id', suffix='_old')
# test_df = join_dfs(test_df, new_hist_trans_agg_cat, left_on='card_id')

In [36]:
test_df.shape, train_df.shape

((123623, 118), (201917, 119))

### Add extra columns like age, total transactions, installments, purchase amount, first buy etc

In [37]:
for df in [train_df, test_df]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df['card_id_count_total'] = df['card_id_count']+df['card_id_count_old']
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['purchase_amount_total_mean'] = df['purchase_amount_mean']+df['purchase_amount_mean_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df['hist_first_buy'] = (df['purchase_date_min_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_min'] - df['first_active_month']).dt.days
    df['hist_last_buy'] = (df['purchase_date_max_old'] - df['first_active_month']).dt.days
    df['new_last_buy'] = (df['purchase_date_max'] - df['first_active_month']).dt.days
    df['purchase_amount_max_total'] = df['purchase_amount_max'] + df['purchase_amount_min_old']
    df['avg_spend_per_transaction'] = df['purchase_amount_total']/df['card_id_total']

### Mark the outliers

In [38]:
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1
train_df['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Adding train days

In [39]:
train_df['days'] = (datetime.date(2018, 2, 1) - train['first_active_month'].dt.date).dt.days

In [40]:
def add_days_feature_interaction(df):
    df['days'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    df['days_feature1'] = df['days'] * df['feature_1']
    df['days_feature2'] = df['days'] * df['feature_2']
    df['days_feature3'] = df['days'] * df['feature_3']
    return df

In [41]:
train_df, test_df = [add_days_feature_interaction(d) for d in [train_df, test_df]]

In [42]:
train_df.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
first_active_monthYear,2017,2017,2016,2017,2017
first_active_monthMonth,6,1,8,9,11
first_active_monthWeek,22,52,31,35,44
first_active_monthDay,1,1,1,1,1


### Target encode the outliers 

In [43]:
train_df.shape, test_df.shape

((201917, 135), (123623, 133))

In [44]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)

In [46]:
train_df.to_feather('train_df_3.695')
test_df.to_feather('test_df_3.695')

In [50]:
# train_df.columns.values